In [1]:
import os 
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
import datetime

import sys
sys.path.append(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration')
from ccf_slice_registration_functions_withUpright import *
from soma_and_fiducial_pins_new import *

%matplotlib inline


Inputs

In [2]:
current_date = datetime.datetime.now().strftime('%Y%m%d')
out = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\ccf_reg_output_{}'.format(current_date) #root for ccf registration output 

os.makedirs(out, exist_ok=True)

Get pins

In [3]:
pins_path = get_soma_and_fiducial_pins()
pins = pd.read_csv(pins_path)

pins['specimen_name'] = pins['specimen_name'].str.strip() #strip any erroneous white space from the start and end fo specimen names
pins = pins[pins['specimen_name'] != 'Point'] #remove one random pin on a cortical slice. 

#pins contains fiducial pins (specimen name ends in a letter) and soma pins (specimen name ends in a number)
#break up pins into fiducial vs soma pin dataframes
fiducials_dict = {}
somas_dict = {}
for i, p in pins.iterrows():
    last_pin_char = p['specimen_name'][-1]
    if last_pin_char.isalpha(): fiducials_dict[i] = p #the last char of the pin name is a letter, so this is a fiducial (not a soma) pin 
    else: somas_dict[i] = p #the last char of the pin name is a number, so this is a soma (not a fiducial) pin
        
somas = pd.DataFrame.from_dict(somas_dict, orient='index').reset_index().drop(['index'], axis=1)
fiducials = pd.DataFrame.from_dict(fiducials_dict, orient='index').reset_index().drop(['index'], axis=1)

slices = fiducials.slide_specimen_id.unique() #get the slices that have fiducials
print('{} slices with fiducials'.format(len(slices)))

{'slide_specimen_id': 1208614358, 'specimen_name': 'Curve'}
{'slide_specimen_id': 1266114320, 'specimen_name': 'Gad2-IRES-Cre;Ai14-670842.06.02.01'}
{'slide_specimen_id': 1374540601, 'specimen_name': 'C57BL6J-742760.03.02.01'}
{'slide_specimen_id': 1262448162, 'specimen_name': 'Sst-IRES-Cre;Ai14-670419.09.03.02'}
342 slices with fiducials


In [4]:
somas.to_csv(os.path.join(out, '20241119_somas.csv'), index=False)
fiducials.to_csv(os.path.join(out, '20241119_fiducials.csv'), index=False)


Filter registration list

In [5]:
# #load a csv and filter the cells and slices to register just those cells
# want_to_register = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\mouse_bg\20241023\20241023_mouse_BG.csv')
# want_to_register['slide_specimen_id'] = want_to_register['Cell Specimen Id'].apply(lambda x: get_id_by_name(get_name_by_id(x).rsplit('.',1)[0]))
# register_slices = want_to_register['slide_specimen_id'].unique().tolist()

# #make note of any cells/slices that we wanted to register that don't have the fiducials to do so
# missing_fiducials = list(set(register_slices) - set(slices))
# print('missing_fiducials: {}'.format(missing_fiducials))

# #update slices/cells to register
# slices = list(set(slices).intersection(set(register_slices))) #only need to update the slices list - then only cells from these slices are registered #TODO is that true??? I don't actually think it is. 

Load CCF

In [6]:
# read in CCF (with fixed headers)
image_directory =  r'\\allen\programs\celltypes\workgroups\mousecelltypes\_UPENN_fMOST\mouse_ccf_fixed_headers_um\average_template'
ccf_file = os.path.join(image_directory, "average_template_10.nii.gz" )
resolution = 10
ccf = sitk.ReadImage( ccf_file )
volume_shape = ccf.GetSize()
z_size = volume_shape[2]*resolution
z_midline = z_size / 2

Transform slices to CCF

In [7]:
#loop through all slices, finding transforms to register and upright them to the ccf 
slices_with_issues = {}
for specimen_id in slices:
    specimen_id = int(specimen_id)

    #get slice name 
    specimen_name = get_name_by_id(specimen_id)
    print(specimen_name)

    try:

        #make folder to store transform for this slice 
        working_directory = os.path.join(out, specimen_name)
        if not os.path.exists(working_directory):
            os.mkdir(working_directory)

        # read in pinning tool output - virtual slice definition and cell ccf locations
        pinning_info = get_pinning(specimen_name)[4]

        # virtual_slice_to_ccf_transform: transform a 3D point (in micron) in the virtual slice to ccf (in micron)
        # ccf_to_virtual_slice_transform: transform a 3D point in ccf (in micron) to virtual slice (in micron)
        # (note: each point can have its own orientation - we are taking the first one only #TODO is this an issue? (swb))
        virtual_slice_to_ccf_transform = initialize_transform( pinning_info['markups'][0]['orientation'] )
        ccf_to_virtual_slice_transform = virtual_slice_to_ccf_transform.GetInverse()

        # write out the virtual_slice_to_ccf_transform
        file = os.path.join(working_directory, 'virtual_slice_to_ccf_transform.txt')
        sitk.WriteTransform( virtual_slice_to_ccf_transform, file )


        # generate a virtual slice from the transforms
        #   virtual_slice_3d is a 3D volume with a single z slice
        #   virtual_slice    is a 2D volume created by extracting out the single slice
        virtual_slice_size = [1250,1250,1] # 3D volume with one slice
        virtual_slice_spacing = ccf.GetSpacing()
        virtual_slice_3d = sitk.Resample(ccf, 
                                        virtual_slice_size, 
                                        virtual_slice_to_ccf_transform, 
                                        sitk.sitkLinear,
                                        [0,0,0], 
                                        virtual_slice_spacing, 
                                        [1,0,0,0,1,0,0,0,1], 
                                        0.0, 
                                        ccf.GetPixelID())#, False )


        virtual_slice = virtual_slice_3d[:,:,0] # 2D image version

        # read in the 2D overview image and associate metadata
        img_info = [list(get_20x_info(specimen_name))]

        odf = pd.DataFrame(img_info, columns = ['specimen_name', 'sub_image_id', 'width', 'height', 'resolution', 'treatment_id'])
        overview_info = odf.loc[0]
        sub_image = odf['sub_image_id'].values[0]
        get_20x_img(sub_image, specimen_name, working_directory)   
        file = os.path.join(working_directory, '{}_overview.jpg'.format(specimen_name))
        overview = sitk.ReadImage( file )
        overview_spacing = [overview_info['resolution'],overview_info['resolution']]
        overview.SetSpacing(overview_spacing)

        # downsample the 2D overview image
        downsampled_overview = sitk.BinShrink( sitk.VectorIndexSelectionCast(overview,0), [25,25])

        # write the downsampled overview to file
        file = os.path.join(working_directory,'downsampled_overview.nii.gz')
        sitk.WriteImage( downsampled_overview, file, True )

        # Read in drawn soma polygons to create matching landmarks set
        df = get_soma_polygons(specimen_id)

        # For each cell
        #  - compute cell soma from polyline in pixels
        #  - convert cell soma location to microns
        #  - join with cell soma location in CCF
        #  - compute cell soma location in virtual slice
        df['center_pixel'] = [compute_center_from_polyline(p) for p in df['poly_coords']]
        df['center_micron'] = [np.multiply(p,overview_spacing) for p in df['center_pixel']]

        jdict = {}
        for m in pinning_info['markups'] :
            jdict[m['name'].strip()] = m['markup']['controlPoints'][0]['position']
            
        df['ccf_coordinate'] = [jdict[p] for p in df['specimen_name']]
        df['virtual_slice_coordinate'] = [ ccf_to_virtual_slice_transform.TransformPoint(p)[:2] for p in df['ccf_coordinate'] ]
        df.to_csv(os.path.join(working_directory, 'alignment_output.csv'), index=False)

        lndmrks = get_landmark_ids(specimen_id)
        ldf = pd.DataFrame()
        for c in lndmrks:
            # print(c)
            this_lndmrk = get_landmark_location(c[0])
            ldf = pd.concat([ldf, this_lndmrk])
            
        ldf['center_pixel'] = [compute_center_from_polyline(p) for p in ldf['poly_coords']]
        ldf['overview_coordinate'] = [np.multiply(p,overview_spacing) for p in ldf['center_pixel']]

        ldf['ccf_coordinate'] = [jdict[p] for p in ldf['specimen_name']]
        ldf['virtual_slice_coordinate'] = [ ccf_to_virtual_slice_transform.TransformPoint(p)[:2] for p in ldf['ccf_coordinate'] ]

        if len(ldf) < 3:
            print('\tnot enough fiducials')
            continue

        # determine if slice was flipped, if so fix virtual to ccf transform 
        flip = False
        if slice_flipped(ldf):
            transform = virtual_slice_to_ccf_transform.GetParameters()
            transform = list(transform)
            idx = -1
            if pinning_info['referenceView'].lower() == 'sagittal': idx = 2 #flip coronal axis in PIL to LPS transform 
            if pinning_info['referenceView'].lower() == 'coronal': idx = 5 #flip saggital axis in RIA to LPS transform 
            if idx > -1: 
                transform[idx] = transform[idx]*-1 #flip axis
                flip = True 
            transform = tuple(transform)
            virtual_slice_to_ccf_transform.SetParameters(transform)

            # write out the virtual_slice_to_ccf_transform
            file = os.path.join(working_directory, 'virtual_slice_to_ccf_transform.txt')
            sitk.WriteTransform( virtual_slice_to_ccf_transform, file )

        # Concatentate landmarks (cell soma + additional) for registration
        # virtual slice (fixed) landmarks
        fixed_landmarks = [tuple(p) for p in df['virtual_slice_coordinate']]
        fixed_landmarks.extend([tuple(p) for p in ldf['virtual_slice_coordinate']] )

        # overview (moving) landmarks
        moving_landmarks = [tuple(p) for p in df['center_micron']]
        moving_landmarks.extend([tuple(p) for p in ldf['overview_coordinate']] )

        # virtual_slice_to_overview_transform: transform a 2D point (in microns) in the virtual slice to overview image (in microns)
        # overview_to_virtual_slice_transform: transform a 3D point in overview image (in microns) to virtual slice (in microns)
        virtual_slice_to_overview_transform = \
            sitk.LandmarkBasedTransformInitializer( sitk.AffineTransform(2), flatten(fixed_landmarks), flatten(moving_landmarks) )
        overview_to_virtual_slice_transform = virtual_slice_to_overview_transform.GetInverse()

        # write out the overview_to_virtual_slice_transform
        file = os.path.join(working_directory,'overview_to_virtual_slice_transform.txt')
        sitk.WriteTransform( overview_to_virtual_slice_transform, file )

        # generate resampled overview image
        resampled_overview = \
            sitk.Resample(downsampled_overview, virtual_slice, virtual_slice_to_overview_transform, \
                        sitk.sitkLinear, 0, downsampled_overview.GetPixelID())

        # write the downsampled overview to file
        file = os.path.join(working_directory,'resampled_overview.nii.gz')
        sitk.WriteImage( resampled_overview, file, True ) 

        # get the upright-only transformation
        overview_to_virtual_slice_upright_transform = get_upright_transformation(downsampled_overview, ccf_to_virtual_slice_transform, virtual_slice_to_overview_transform, flip)
        moving_landmarks_upright = [overview_to_virtual_slice_upright_transform.TransformPoint(xy) for xy in moving_landmarks]

        # upright downsampled overview and save to file
        upright_overview = sitk.Resample(downsampled_overview, overview_to_virtual_slice_upright_transform, sitk.sitkLinear, 0.0, downsampled_overview.GetPixelID())
        file = os.path.join(working_directory,'upright_overview.nii.gz')
        sitk.WriteImage( upright_overview, file, True ) 

        #save the upright transform to use on morphologies 
        file = os.path.join(working_directory,'overview_to_virtual_slice_upright_transform.txt')
        sitk.WriteTransform( overview_to_virtual_slice_upright_transform, file )


        fig, axes = plt.subplots(nrows=1,ncols=4,figsize=(20,5)) 
        visualize_landmarks(downsampled_overview, axes[0], moving_landmarks)
        axes[0].set_title('20x with fiducials')
        visualize_landmarks( virtual_slice, axes[1], fixed_landmarks)
        axes[1].set_title('virtual slice with fiducials')
        visualize_landmarks( resampled_overview, axes[2], fixed_landmarks)
        axes[2].set_title('resampled 20x')
        visualize_landmarks( upright_overview, axes[3], moving_landmarks_upright)
        axes[3].set_title('upright 20x')
        plt.savefig(os.path.join(working_directory, 'transformation_overview.jpg'))
        # plt.show() 
        plt.clf()


    except: 
        print('\tissue with this slice {}'.format(specimen_id))
        slices_with_issues[specimen_name] = 'issue with this slice'
        continue


Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-561250.06.06
Htr3a-Cre_NO152;Ai14-672490.06.03
Vip-IRES-Cre;Ai14-548766.08.06
Nos1-CreERT2;Sst-IRES-FlpO;Ai65-553472.08.06
Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-561244.07.05
	issue with this slice 1075814440
Sst-IRES-Cre;Ai14-554002.07.06
Htr3a-Cre_NO152;Ai14-672490.08.04
Chat-IRES-Cre-neo;Ai14-623124.07.03
Sst-IRES-Cre;Ai14-572054.06.10
Htr3a-Cre_NO152;Ai14-673660.08.03
Ndnf-IRES2-dgCre;Ai14-597333.09.01
Chat-IRES-Cre-neo;Ai14-627361.08.03
Ndnf-IRES2-dgCre;Ai14-659660.06.10
Penk-IRES2-Cre-neo;Slc17a6-IRES2-FlpO-557544-Ai65.05.09
Pvalb-IRES-Cre;Ai14-553660.05.06
Vip-IRES-Cre;Ai14-633751.14.04
C57BL6J-576333.08.04
Pvalb-IRES-Cre;Ai14-601230.11.01
Rbp4-Cre_KL100;Ai14-601911.05.03
C57BL6J-622938.07.03
Pvalb-IRES-Cre;Ai14-553660.08.06
Htr3a-Cre_NO152;Ai14-609164.05.03


C:\Users\sarah.wallingbell\AppData\Local\Temp\ipykernel_48184\3774322932.py:164: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(nrows=1,ncols=4,figsize=(20,5))


Pvalb-IRES-Cre;Ai14-577972.06.03
Pvalb-IRES-Cre;Ai14-577972.05.04
Gad2-IRES-Cre;Ai14-600203.07.04
C57BL6J-631344.05.03
Vip-IRES-Cre;Ai14-579327.07.04
Vip-IRES-Cre;Ai14-579326.07.03
C57BL6J-581848.07.04
Pvalb-IRES-Cre;Ai14-609302.05.04
Pvalb-IRES-Cre;Ai14-609302.08.04
C57BL6J-665266.07.06
Pvalb-IRES-Cre;Ai14-609902.08.03
Chat-IRES-Cre-neo;Ai14-591848.07.03
Chat-IRES-Cre-neo;Ai14-582721.07.03
C57BL6J-618127.06.03
Pvalb-IRES-Cre;Ai14-609903.07.04
Pvalb-IRES-Cre;Ai14-610299.08.04
Rorb-IRES2-Cre-neo;Ai14-556727.06.06
Htr3a-Cre_NO152;Ai14-617732.06.03
C57BL6J-631344.08.04
Pvalb-IRES-Cre;Ai14-621141.13.09
Pvalb-IRES-Cre;Ai14-621142.14.09
Pvalb-IRES-Cre;Ai14-621143.05.03
C57BL6J-581849.06.03
Pvalb-IRES-Cre;Ai14-621143.08.04
Pvalb-IRES-Cre;Ai14-627393.07.03
Pvalb-IRES-Cre;Ai14-629895.13.03
C57BL6J-618128.07.03
C57BL6J-581849.07.03
Pvalb-IRES-Cre;Ai14-635532.12.04
Chat-IRES-Cre-neo;Ai14-628761.08.03
Slc17a8-iCre;Slc32a1-IRES2-FlpO;Ai65-560686.12.09
Vip-IRES-Cre;Ai14-579325.07.04
Chat-IRES-Cre-ne

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

...

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

In [8]:

slices_with_issues_df = pd.DataFrame.from_dict(slices_with_issues.items())
slices_with_issues_df.to_csv(os.path.join(out, 'slices_with_issues_{}.csv'.format(current_date)), index=False)


Transform cell morphologies to CCF

In [9]:
somas

,slide_specimen_id,specimen_name,x,y,z,structure_id,structure_acronym
0,1254192732,C57BL6J-667412.13.10.01,6725.000000,3807.984131,6180.958496,362,MD
1,1254192732,C57BL6J-667412.13.10.02,6725.000000,3685.511719,6276.875000,362,MD
2,1254192732,C57BL6J-667412.13.10.03,6725.000000,3607.036133,6302.043457,362,MD
3,1189282403,C57BL6J-634112.05.01.01,8846.905344,1215.824000,2322.970703,821,VISp2/3
4,1189282403,C57BL6J-634112.05.01.02,8698.307617,1624.090698,2576.191895,778,VISp5
...,...,...,...,...,...,...,...
4336,1381879715,Slc17a8-IRES2-Cre-neo;Ai14-744792.04.09.01,9400.000000,3390.569580,5675.099609,872,DR
4337,1381879715,Slc17a8-IRES2-Cre-neo;Ai14-744792.04.09.02,9400.000000,3500.603760,5664.691895,872,DR
4338,1225979537,Slc17a6-IRES-Cre;Ai14-651168.10.06.01,10347.208008,4123.347168,4529.000000,867,PB
4339,1225979537,Slc17a6-IRES-Cre;Ai14-651168.10.06.02,10288.272461,4340.153320,4529.000000,867,PB


In [10]:
registered_cells = []
uprighted_cells = []
cells_with_issues = {}

for idx, cell in somas.iterrows():
    sp_name = cell['specimen_name']
    sl_name = sp_name.rsplit('.', 1)[0]

    print(sp_name)

    try:
        sp_id = get_id_by_name(sp_name)

        #check if there's a transform for this slice 
        slice_path = os.path.join(out, sl_name)
        if not os.path.isfile(os.path.join(slice_path, 'overview_to_virtual_slice_transform.txt')): continue #go on to next cell 
        if not os.path.isfile(os.path.join(slice_path, 'virtual_slice_to_ccf_transform.txt')): continue #go on to next cell 
        if not os.path.isfile(os.path.join(slice_path, 'alignment_output.csv')): continue #go on to next cell 


        #get soma loc in 20x 
        alignment_output = pd.read_csv(os.path.join(slice_path, 'alignment_output.csv'))
        this_cell_alignment = alignment_output.query("draw_type == 'Soma'")[alignment_output.specimen_name == sp_name]
        if len(this_cell_alignment) == 0: 
            cells_with_issues[sp_name] = 'No soma pin'
            print('\t no soma pin')
            continue
        if len(this_cell_alignment) > 1: 
            cells_with_issues[sp_name] = 'Multiple soma pins'
            print('\t multiple soma pins')
            continue
        lims_soma = this_cell_alignment['center_micron'].values
        lims_soma = lims_soma[0][1:-2].split(' ')
        lims_soma = [float(i) for i in lims_soma if len(i) > 0]


        #make folder to save registered swcs
        swc_path = os.path.join(slice_path, 'SWC')
        if not os.path.exists(swc_path): os.mkdir(swc_path)


        #load affine transforms
        overview_to_virtual_slice_transform =           sitk.ReadTransform(os.path.join(slice_path, 'overview_to_virtual_slice_transform.txt'))
        overview_to_virtual_slice_upright_transform =   sitk.ReadTransform(os.path.join(slice_path, 'overview_to_virtual_slice_upright_transform.txt'))
        virtual_slice_to_ccf_transform =                sitk.ReadTransform(os.path.join(slice_path, 'virtual_slice_to_ccf_transform.txt'))


        #register manual swc 
        try: 
            swc_name = '{}'.format(sp_id)
            lims_path = list(get_swc_from_lims(str(sp_id)))[1]
            lims_path = edit_path(lims_path)

            #register to CCF
            morph = to_dict(lims_path)
            register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                        overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform,
                        resolution, volume_shape, z_midline)
            registered_cells = registered_cells + [sp_name+'_manual']
            
            #upright 
            morph = to_dict(lims_path)
            upright_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                        overview_to_virtual_slice_upright_transform, virtual_slice_to_ccf_transform,
                        resolution, volume_shape, z_midline)
            uprighted_cells = uprighted_cells + [sp_name+'_manual']
            
        except: print('\t\tno manual swc to register')


        autotrace_registered = False
        #register autotrace post processed step 14 swc
        try: 
            pp14_path = get_autotrace_pp_path(sp_id, 14)
            swc_name = pp14_path.rsplit('\\',1)[1].split('.swc',1)[0]

            #register to CCF
            morph = to_dict(pp14_path)
            morph = convert_pixel_to_um_dictnrn(morph, sp_id)
            register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                    overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform,
                    resolution, volume_shape, z_midline)
            registered_cells = registered_cells + [sp_name+'_pp14']

            #upright 
            morph = to_dict(pp14_path)
            morph = convert_pixel_to_um_dictnrn(morph, sp_id)
            upright_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                        overview_to_virtual_slice_upright_transform, virtual_slice_to_ccf_transform,
                        resolution, volume_shape, z_midline)
            uprighted_cells = uprighted_cells + [sp_name+'_manual']

            autotrace_registered = True
        except: print('\t\tno pp14 autotrace swc to register')

        
        #register raw autotrace swc if no post processed version 
        if not autotrace_registered:
            try: 
                raw_path = get_autotrace_raw_path(sp_id)
                swc_name = raw_path.rsplit('\\',1)[1].split('.swc',1)[0]

                #register to CCF
                morph = to_dict(raw_path)
                morph = convert_pixel_to_um_dictnrn(morph, sp_id)
                register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                        overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform,
                        resolution, volume_shape, z_midline)
                registered_cells = registered_cells + [sp_name+'_raw']

                #upright 
                morph = to_dict(raw_path)
                morph = convert_pixel_to_um_dictnrn(morph, sp_id)
                upright_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                            overview_to_virtual_slice_upright_transform, virtual_slice_to_ccf_transform,
                            resolution, volume_shape, z_midline)
                uprighted_cells = uprighted_cells + [sp_name+'_manual']

            except: print('\t\tno raw autotrace swc to register')

    except: 
        cells_with_issues[sp_name] = 'Issue with this cell'
        continue

C57BL6J-667412.13.10.01
C57BL6J-667412.13.10.02
C57BL6J-667412.13.10.03
C57BL6J-634112.05.01.01
C57BL6J-634112.05.01.02
C57BL6J-634112.05.01.03
C57BL6J-634221.05.01.01
Rbp4-Cre_KL100;Ai14-659000.10.06.02
Rbp4-Cre_KL100;Ai14-659000.10.06.03
Rbp4-Cre_KL100;Ai14-659000.10.06.04
Pvalb-IRES-Cre;Ai14-593556.12.02.02
C57BL6J-634222.04.01.01
Rorb-IRES2-Cre-neo;Ai14-604947.14.02.01
Pvalb-IRES-Cre;Ai14-593558.04.01.01
Pvalb-IRES-Cre;Ai14-593558.04.01.02
Pvalb-IRES-Cre;Ai14-593558.04.01.03
Pvalb-IRES-Cre;Ai14-593558.04.01.04
Pvalb-IRES-Cre;Ai14-593558.04.02.01
Pvalb-IRES-Cre;Ai14-593558.04.02.02
Pvalb-IRES-Cre;Ai14-593558.04.02.04
Vip-IRES-Cre;Ai14-594741.14.02.01
Vip-IRES-Cre;Ai14-594741.14.02.02
Vip-IRES-Cre;Ai14-594741.14.02.03
C57BL6J-625941.04.02.01
Slc17a6-IRES-Cre;Ai14-634163.11.02.01
Slc17a6-IRES-Cre;Ai14-634163.11.02.02
Slc17a6-IRES-Cre;Ai14-634163.11.02.03
Gad2-IRES-Cre;Ai14-626576.03.01.02
Gad2-IRES-Cre;Ai14-626576.03.01.04
Gad2-IRES-Cre;Ai14-626576.03.01.06
Gad2-IRES-Cre;Ai14-626576.0

c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-pack

Htr3a-Cre_NO152;Ai14-672490.08.04.03
		no manual swc to register
		no pp14 autotrace swc to register
		no raw autotrace swc to register
Gad2-IRES-Cre;Ai14-600202.11.02.01
Gad2-IRES-Cre;Ai14-600202.11.02.02
Gad2-IRES-Cre;Ai14-600202.11.02.03
Slc17a6-IRES-Cre;Ai14-599403.11.01.01
Slc17a6-IRES-Cre;Ai14-599403.11.01.02
Slc17a6-IRES-Cre;Ai14-599403.11.01.03
Gad2-IRES-Cre;Ai14-600204.10.01.01
Gad2-IRES-Cre;Ai14-600204.10.01.02
C57BL6J-628260.02.01.01
C57BL6J-628260.02.01.02
C57BL6J-628261.03.01.01
C57BL6J-628261.03.01.02
C57BL6J-628261.03.01.03
Pvalb-IRES-Cre;Ai14-606194.10.02.02
Pvalb-IRES-Cre;Ai14-606194.10.02.03
Pvalb-IRES-Cre;Ai14-606194.10.02.04
Pvalb-IRES-Cre;Ai14-606194.10.02.05
Pvalb-IRES-Cre;Ai14-606194.10.02.01
C57BL6J-628261.04.01.01
C57BL6J-628261.04.01.02
C57BL6J-628261.04.01.03
Rorb-IRES2-Cre-neo;Ai14-598708.09.01.01
Rorb-IRES2-Cre-neo;Ai14-598708.09.01.02
Sst-IRES-Cre;Ai14-508748.07.06.01
Sst-IRES-Cre;Ai14-508748.07.06.02
Sst-IRES-Cre;Ai14-508748.07.06.03
Rorb-IRES2-Cre-neo;Ai

c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-pack

C57BL6J-613674.13.09.01
C57BL6J-613674.13.09.02
C57BL6J-613674.13.09.03
Slc17a6-IRES-Cre;Ai14-591727.12.09.03
Slc17a6-IRES-Cre;Ai14-616582.13.09.01
Slc17a6-IRES-Cre;Ai14-616582.13.09.02
Slc17a6-IRES-Cre;Ai14-616582.13.09.03
Chat-IRES-Cre-neo;Ai14-606032.07.03.01
Chat-IRES-Cre-neo;Ai14-606032.07.03.02
Chat-IRES-Cre-neo;Ai14-606032.07.03.03
C57BL6J-640171.10.01.01
C57BL6J-640171.10.01.02
Pvalb-IRES-Cre;Ai14-604427.10.01.01
Sst-IRES-Cre;Ai14-572054.06.10.01
Sst-IRES-Cre;Ai14-572054.06.10.02
		no manual swc to register
		no pp14 autotrace swc to register
		no raw autotrace swc to register
Pvalb-IRES-Cre-591411.07.09.01
Pvalb-IRES-Cre-591411.07.09.02
Pvalb-IRES-Cre-591411.07.09.03
Pvalb-IRES-Cre-591411.07.09.04
Sst-IRES-Cre-599593.10.09.01
Sst-IRES-Cre-599593.10.09.02
Slc17a6-IRES-Cre;Ai14-612219.06.03.01
Slc17a6-IRES-Cre;Ai14-612219.06.03.02
Slc17a6-IRES-Cre;Ai14-612219.06.03.03
Slc17a6-IRES-Cre;Ai14-612219.06.03.04
Slc17a6-IRES-Cre;Ai14-606635.12.02.01
C57BL6J-639264.11.01.01
C57BL6J-6392

...

c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-pack

Rbp4-Cre_KL100;Ai14-659002.11.06.01
	 no soma pin
Rbp4-Cre_KL100;Ai14-659002.11.06.02
Ndnf-IRES2-dgCre;Ai14-659661.11.06.01
Ndnf-IRES2-dgCre;Ai14-659661.11.06.02
	 no soma pin
Ndnf-IRES2-dgCre;Ai14-659661.11.06.03
	 no soma pin
Pvalb-IRES-Cre;Ai14-650959.11.06.01
	 no soma pin
Pvalb-IRES-Cre;Ai14-650959.11.06.02
	 no soma pin
Pvalb-IRES-Cre;Ai14-650959.11.06.03
	 no soma pin
Pvalb-IRES-Cre;Ai14-650959.11.06.04


c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-pack

Rbp4-Cre_KL100;Ai14-658998.09.06.01
Rbp4-Cre_KL100;Ai14-658998.09.06.02
	 no soma pin
Rbp4-Cre_KL100;Ai14-658998.09.06.03
Rbp4-Cre_KL100;Ai14-658998.09.06.04
	 no soma pin
Slc17a6-IRES-Cre;Ai14-651169.10.06.01
	 no soma pin
Slc17a6-IRES-Cre;Ai14-651169.10.06.02
	 no soma pin
Slc17a6-IRES-Cre;Ai14-651169.10.06.03
Slc17a6-IRES-Cre;Ai14-651169.10.06.04
Slc17a6-IRES-Cre;Ai14-670828.09.06.01
Slc17a8-IRES2-Cre-neo;Ai14-744792.04.09.01
Slc17a8-IRES2-Cre-neo;Ai14-744792.04.09.02
	 no soma pin
Slc17a6-IRES-Cre;Ai14-651168.10.06.01
	 no soma pin
Slc17a6-IRES-Cre;Ai14-651168.10.06.02
Slc17a6-IRES-Cre;Ai14-651168.10.06.03


c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-packages\scipy\interpolate\_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\sarah.wallingbell\AppData\Local\anaconda3\envs\morph_utils_v3\lib\site-pack

In [11]:
cells_with_issues_df = pd.DataFrame.from_dict(list(cells_with_issues.items())) 
cells_with_issues_df.to_csv(os.path.join(out, 'cells_with_issues_{}.csv'.format(current_date)), index=False)

In [12]:
registered_cells_df = pd.DataFrame(registered_cells)
registered_cells_df.to_csv(os.path.join(out, 'registered_cells_{}.csv'.format(current_date)), index=False)